In [129]:
from sklearn.datasets import load_diabetes
import numpy as np

X, y = load_diabetes(return_X_y=True)

Example with no prior training and successive tuning

In [130]:
from CRR import ConformalRidgeRegressor

cp = ConformalRidgeRegressor(autotune=True)

Err = 0

predictions = np.empty((X.shape[0], 5))
for i, (object, label) in enumerate(zip(X, y)):
    Gamma = cp.predict(object, epsilon=0.1, bounds='both')
    width = cp.width(Gamma)
    cp.learn_one(object, label)  
    if i > 1 and i % 10 == 0:
        # Autotune ridge parameter every 10:th step. Not neccesary for validity, but could improve efficiency
        cp.tune_ridge_parameter()
    err = cp.err(Gamma, label)
    Err += err
    predictions[i, 0] = Gamma[0] # Lower 
    predictions[i, 1] = Gamma[1] # Upper 
    predictions[i, 2] = err # Error
    predictions[i, 3] = label # True label
    predictions[i, 4] = width # True label

/home/johan/Documents/PhD/PREMACOP/OnlineConformalPrediction/CRR.py:158: UserWarning: Significance level epsilon is too small for training set. Need at least 20 examples. Increase or add more examples
  warnings.warn(f'Significance level epsilon is too small for training set. Need at least {int(np.ceil(2/epsilon))} examples. Increase or add more examples')
/home/johan/Documents/PhD/PREMACOP/OnlineConformalPrediction/CRR.py:284: UserWarning: The matrix X is rank deficient. Condition number: 1.8377100903521824e+17. Autotuning ridge prarmeter.
  warnings.warn(f'The matrix X is rank deficient. Condition number: {np.linalg.cond(M)}. Autotuning ridge prarmeter.')


In [131]:
cp.a

0.49224380006203494

In [132]:
Err / X.shape[0]

0.09502262443438914

In [133]:
predictions[100:,4].mean()

200.07827416759022

In [134]:
np.isclose(cp.XTXinv, np.linalg.inv(cp.X.T @ cp.X + cp.a * cp.Id)).all()

True

Example with initial training set and no successive tuning

In [135]:
from CRR import ConformalRidgeRegressor

cp = ConformalRidgeRegressor(autotune=True)

initial_training_size = 100
X_train = X[:initial_training_size]
y_train = y[:initial_training_size]

X_run = X[initial_training_size:]
y_run = y[initial_training_size:]

cp.learn_initial_training_set(X_train, y_train)

Err = 0

predictions = np.empty((X_run.shape[0], 5))
for i, (object, label) in enumerate(zip(X_run, y_run)):
    Gamma = cp.predict(object, epsilon=0.1, bounds='both')
    width = cp.width(Gamma)
    cp.learn_one(object, label)  
    err = cp.err(Gamma, label)
    Err += err
    predictions[i, 0] = Gamma[0] # Lower 
    predictions[i, 1] = Gamma[1] # Upper 
    predictions[i, 2] = err # Error
    predictions[i, 3] = label # True label
    predictions[i, 4] = width # True label

In [136]:
cp.a

1.1098525693753387

In [137]:
Err / X[initial_training_size:].shape[0]

0.09064327485380116

In [138]:
predictions[:,4].mean()

204.86341348767908

In [139]:
np.isclose(cp.XTXinv, np.linalg.inv(cp.X.T @ cp.X + cp.a * cp.Id)).all()

True